In [ ]:
from utils import load_all_data, get_teg_winners, get_teg_rounds, datawrapper_table_css
import pandas as pd

import warnings
warnings.filterwarnings('ignore', category=Warning, module='streamlit')

all_data = load_all_data(exclude_incomplete_tegs=True, exclude_teg_50=True)
filtered_data = all_data.copy()

df_all = get_teg_winners(filtered_data)
#df_all.columns.to_series().to_clipboard(sep=',', index=False, header=False)



In [ ]:


df = df_all[['TEG','TEG Trophy']]

summary_df = df.groupby('TEG Trophy').agg(
    Wins=('TEG', 'count'),  # New column name 'count' with the count of 'TEG'
    TEGs=('TEG', list)     # New column name 'TEGs' with a list of 'TEG's
).reset_index()

summary_df.rename(columns={'TEG Trophy': 'Player'}, inplace=True)
sorted_summary_df = summary_df.sort_values(by='Wins', ascending=False)
sorted_summary_df['TEGs'] = sorted_summary_df['TEGs'].apply(', '.join)

print(sorted_summary_df)


In [10]:
comps = ['TEG Trophy', 'Green Jacket', 'HMM Wooden Spoon']

# Loop through each comp
for column in comps:
    # Print a header for each output
    print(f"--- Summarizing by '{column}' ---")

    # Select only the relevant columns for the current iteration
    df = df_all[['TEG', column]].dropna()

    summary_df = df.groupby(column).agg(
        Wins=('TEG', 'count'),
        TEGs=('TEG', list)
    ).reset_index()

    summary_df.rename(columns={column: 'Player'}, inplace=True)
    
    sorted_summary_df = summary_df.sort_values(by='Wins', ascending=False)
    
    sorted_summary_df['TEGs'] = sorted_summary_df['TEGs'].apply(', '.join)

    print(sorted_summary_df)
    print("\n")

--- Summarizing by 'TEG Trophy' ---
           Player  Wins                           TEGs
2  Gregg WILLIAMS     4    TEG 5, TEG 6, TEG 8, TEG 15
5       Jon BAKER     4  TEG 3, TEG 11, TEG 13, TEG 17
1    David MULLIN     3           TEG 2, TEG 4, TEG 14
4  John PATTERSON     2                  TEG 9, TEG 12
0      Alex BAKER     1                         TEG 10
3    Henry MELLER     1                          TEG 7
6  Stuart NEUMANN     1                         TEG 16


--- Summarizing by 'Green Jacket' ---
            Player  Wins                                               TEGs
0     David MULLIN     9  TEG 2, TEG 3, TEG 4, TEG 6, TEG 7, TEG 8, TEG ...
2        Jon BAKER     4                     TEG 11, TEG 12, TEG 13, TEG 17
1   Gregg WILLIAMS     2                                     TEG 15, TEG 16
3  Stuart NEUMANN*     1                                              TEG 5


--- Summarizing by 'HMM Wooden Spoon' ---
           Player  Wins                           TEGs
1    

In [ ]:
import os
os.chdir('../streamlit')

In [8]:
from io import StringIO
import pandas as pd
from utils import load_all_data, get_teg_winners, get_teg_rounds, datawrapper_table_css
import warnings
warnings.filterwarnings('ignore', category=Warning, module='streamlit')


def summarize_by_column(df, column_name):
    """
    Summarizes a DataFrame by a specified column, counting TEGs and listing them.

    Args:
        df (pd.DataFrame): The input DataFrame.
        column_name (str): The column to group by.

    Returns:
        pd.DataFrame: A summarized and sorted DataFrame.
    """
    # Select only the relevant columns and drop rows with NaN values
    temp_df = df[['TEG', column_name]].dropna(subset=[column_name])

    # Group by the specified column and aggregate
    summary_df = temp_df.groupby(column_name).agg(
        Wins=('TEG', 'count'),
        TEGs=('TEG', list)
    ).reset_index()

    # Rename the grouping column to 'Player'
    summary_df.rename(columns={column_name: 'Player'}, inplace=True)
    
    # Sort by the number of wins in descending order
    sorted_summary_df = summary_df.sort_values(by='Wins', ascending=False)
    
    # Convert the list of TEGs to a comma-separated string
    sorted_summary_df['TEGs'] = sorted_summary_df['TEGs'].apply(', '.join)
    
    return sorted_summary_df


# Helper function to convert a comma-separated string of numbers to a range string
def numbers_to_ranges(number_string):
    # Handles empty or invalid input
    if not number_string:
        return ""
    
    # Split the string and convert to a sorted list of integers
    numbers = sorted([int(x) for x in number_string.split(', ')])
    
    ranges = []
    i = 0
    while i < len(numbers):
        start = numbers[i]
        j = i + 1
        while j < len(numbers) and numbers[j] == numbers[j-1] + 1:
            j += 1
        end = numbers[j-1]
        
        if start == end:
            ranges.append(str(start))
        else:
            ranges.append(f"{start}-{end}")
        
        i = j
    
    return ', '.join(ranges)

all_data = load_all_data(exclude_incomplete_tegs=True, exclude_teg_50=True)
filtered_data = all_data.copy()

df_all = get_teg_winners(filtered_data)

comps = ['TEG Trophy', 'Green Jacket', 'HMM Wooden Spoon']
# comp = comps[0]

for comp in comps:

    summary_table = summarize_by_column(df_all, comp)
    summary_table['TEGs'] = summary_table['TEGs'].str.replace('TEG ', '')
    summary_table['TEGs'] = summary_table['TEGs'].apply(numbers_to_ranges)

    print(f"\n========== | {comp} | ==========")
    print(summary_table)
    print(f"=========================================")


========== | TEG Trophy | ==========
           Player  Wins           TEGs
2  Gregg WILLIAMS     4     5-6, 8, 15
5       Jon BAKER     4  3, 11, 13, 17
1    David MULLIN     3       2, 4, 14
4  John PATTERSON     2          9, 12
0      Alex BAKER     1             10
3    Henry MELLER     1              7
6  Stuart NEUMANN     1             16

========== | Green Jacket | ==========
            Player  Wins           TEGs
0     David MULLIN     9  2-4, 6-10, 14
2        Jon BAKER     4      11-13, 17
1   Gregg WILLIAMS     2          15-16
3  Stuart NEUMANN*     1              5

========== | HMM Wooden Spoon | ==========
           Player  Wins      TEGs
1    David MULLIN     4  5, 11-13
0      Alex BAKER     3  7, 15-16
6  Stuart NEUMANN     3    3, 8-9
2  Gregg WILLIAMS     2     4, 17
3    Henry MELLER     2      2, 6
4  John PATTERSON     1        10
5       Jon BAKER     1        14
